# Part Two
## 1.Loading NSL_KDD DataSet

In [280]:
import pandas as pd # importig pandas to laod the file into data frame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
columns = [ "duration","protocol_type","service","flag","src_bytes","dst_bytes",
"land","wrong_fragment","urgent","hot","num_failed_logins",
"logged_in","num_compromised","root_shell","su_attempted",
"num_root","num_file_creations","num_shells","num_access_files",
"num_outbound_cmds","is_host_login","is_guest_login","count",
"srv_count","serror_rate","srv_serror_rate","rerror_rate",
"srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate",
"dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
"dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate",
"dst_host_srv_serror_rate","dst_host_rerror_rate",
"dst_host_srv_rerror_rate","label" ,"difficulty" ]

data = pd.read_csv('data.txt', header=None)
data.columns = columns

print(data.head()) # printing the first 5 rows of the data frame


   duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0         0           tcp  ftp_data   SF        491          0     0   
1         0           udp     other   SF        146          0     0   
2         0           tcp   private   S0          0          0     0   
3         0           tcp      http   SF        232       8153     0   
4         0           tcp      http   SF        199        420     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                    0.17   
1               0       0    0  ...                    0.00   
2               0       0    0  ...                    0.10   
3               0       0    0  ...                    1.00   
4               0       0    0  ...                    1.00   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.03                         0.17   
1                    0.60                         0.88   
2             

In [281]:
data.info() #displaying the data information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

In [282]:
data['label'].value_counts() # displaying all the normal samples and attack samples counts

label
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: count, dtype: int64

## 1. Why do we set random seeds? What is reproducibility in machine learning?
- In machine learning, we set random seeds to ensure reproducibility, meaning the same code and data always produce the same results

## 2. How many samples are in the dataset?
- its 125973 from data.info()

## 3. How many features are present (excluding label and difficulty)?
- there are 41 total features(excluding label and difficulty) because from data.info() we get 43

## 4. What is the distribution between normal samples and attacks?
- Normal samples: 67,343 (53.4% of dataset)
- Attack samples: 58,630 (46.6% of dataset)
- Total: 125,973 samples
- This is a ~53:47 ratio, which is pretty good for machine learning (not heavily imbalanced)
## 5. Why is this distribution important for model training?

- This distribution is important because the model learns patterns from the data, so if the training data doesn’t represent the real-world distribution, the model won’t perform well on new data.

# Part 3

In [283]:
data.drop(['difficulty'], axis=1, inplace=True) # dropping the difficulty column as it is not needed

In [284]:
X_features = data.drop(['label'], axis=1) # features data frame without the label column
Y_labels = data['label'] # labels data frame with only the label column


In [285]:
X_features = pd.get_dummies(X_features, columns=['protocol_type', 'service', 'flag'])
# One-hot encoding for categorical features


In [286]:
Y_labels = data['label'].apply(lambda x: 0 if x == 'normal' else 1)
# converting labels to binary values: 0 for normal, 1 for attack

In [287]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features) # fit transform to scale the features

X_scaled = pd.DataFrame(X_scaled, columns=X_features.columns) # converting back to data frame

In [288]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_labels, test_size=0.2, random_state=42) 
# splitting the data into training and testing sets with 80% training and 20% testing



## 6. What encoding technique do you use for categorical variables? Why?
- we use One-Hot Encoding (via pd.get_dummies()) because Neural networks need numbers, not text while avoiding false numerical relationships between categories.

## 7. How many features do you have after encoding? Why did the number increase?

- we have 122 features because One-hot encoding replaces each categorical column with multiple binary columns (one per unique value). 

## 8. What type of classification problem is this (binary, multi-class, multi-label)?

- its a Binary Classification problem becasuse we encoded the labels as 0 (normal) or 1 (attack), creating only two possible classes.

## 9. Why is feature scaling important for neural networks?
- Without scaling, features with large values (e.g., src_bytes = 50,000) dominate the gradient updates over small-value features (e.g., serror_rate = 0.05), causing the model to ignore smaller features.

## 10. How many samples are in the training and testing sets?
- X_Train:100778
- X_Test: 25195
- Y_Train: 100778
- Y_Test:25195

# Part 4

In [ ]:
# definition of the build function

def build_model(n_hidden_layers, n_neurons, learning_rate, dropout_rate):
    model = Sequential()
    for _ in range(n_hidden_layers):
        model.add(Dense(n_neurons, activation='relu', input_shape=(122,)))# adding hidden layers with relu activation function
        model.add(Dropout(dropout_rate))# adding dropout layer to prevent overfitting
    model.add(Dense(1, activation='sigmoid'))  # output layer for binary classification
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    #compiling the model with adam optimizer and binary crossentropy loss function that is suitable for binary classification and the adam optimizer is an efficient optimization algorithm that adapts the learning rate during training.
    return model


## 12. Why use sigmoid activation for the output layer?
- We use sigmoid activation for the output layer in binary classification because it squashes the output to a value between 0 and 1, which can be interpreted as the probability of the positive class.

## 13. What is the purpose of dropout layers?
- The purpose of dropout layers is to prevent overfitting in neural networks.

## 14. Compare the two architectures:


| Model           | Hidden Layers | Neurons per Layer | Trainable Parameters | Compression (Input → First Hidden) |
|-----------------|---------------|-----------------|--------------------|----------------------------------|
| Shallow Network | 1             | 4               | 497                | 122 → 4 ≈ 31                     |
| Deep Network    | 3             | 32              | 6071               | 122 → 32 ≈ 3.8                   |

## Notes
- **Trainable parameters calculation:**  
  - Dense layer: `(inputs × neurons) + neurons (biases)`  
  - Shallow hidden layer: (122×4) + 4 = 492  
  - Shallow output layer: (4×1) + 1 = 5 → Total = 492 + 5 = 497  
  - Deep first hidden: (122×32) + 32 = 3936  
  - Deep second hidden: (32×32) + 32 = 1056  
  - Deep third hidden: (32×32) + 32 = 1056  
  - Deep output: (32×1) + 1 = 33 → Total = 3936+1056+1056+33 = 6071  
- **Compression ratio:**  
  - Shallow: 122 input features → 4 neurons → 122 ÷ 4 ≈ 31  
    (high compression → lots of input info squeezed)  
  - Deep: 122 input features → 32 neurons → 122 ÷ 32 ≈ 3.8  
    (less compression → preserves more info for deeper layers)

## 15. Predict which model will perform better and explain why.

The deep network (Model 2) will likely perform better.

Why:

- It has more hidden layers and neurons, so it can learn more complex patterns in the data.

- Dropout prevents overfittin.

- Smaller learning rate and batch size allow stable and precise training.